In [ ]:
!pip install qutip matplotlib numpy

# Imports
import numpy as np
import matplotlib.pyplot as plt
from qutip import destroy, basis, sesolve, expect, tensor, qeye, squeeze, expm, ptrace, wigner

In [ ]:
##Project 1: Squeezed-Vacuum Dynamics

In [ ]:
# Parameters
N = 20                # Fock basis size
r = 0.8               # squeezing strength
t_list = np.linspace(0, 2, 200)

# Operators
a = destroy(N)
H_single = 1j * r * (a.dag()**2 - a**2)
psi0 = basis(N, 0)

# Solve dynamics
res1 = sesolve(H_single, psi0, t_list, [])

# Quadratures
X = (a + a.dag())/np.sqrt(2)
P = (a - a.dag())/(1j*np.sqrt(2))
varX = [expect(X**2, st) - expect(X, st)**2 for st in res1.states]
varP = [expect(P**2, st) - expect(P, st)**2 for st in res1.states]

In [ ]:
# Plot Single‑Mode Squeezing
plt.figure(figsize=(6,4))
plt.plot(t_list, varX, label='Var(X)')
plt.plot(t_list, varP, label='Var(P)')
plt.xlabel('Time')
plt.ylabel('Variance')
plt.title('Single‑Mode Squeezed Vacuum Dynamics')
plt.legend(); plt.grid(True)
plt.savefig('single_mode_squeezing.png', dpi=300)
plt.show()

In [ ]:
# Two‑mode parameters
N = 15
r2 = 0.8
t_list2 = np.linspace(0, 2, 200)
a, b = destroy(N), destroy(N)
H_two = 1j * r2 * (tensor(a.dag(), b.dag()) - tensor(a, b))
psi0_2 = tensor(basis(N,0), basis(N,0))

res2 = sesolve(H_two, psi0_2, t_list2, [])
X1 = (tensor(a, qeye(N)) + tensor(a.dag(), qeye(N)))/np.sqrt(2)
X2 = (tensor(qeye(N), b) + tensor(qeye(N), b.dag()))/np.sqrt(2)
varX1 = [expect(X1**2, st) - expect(X1, st)**2 for st in res2.states]
varX2 = [expect(X2**2, st) - expect(X2, st)**2 for st in res2.states]

In [ ]:
# Plot Two‑Mode Squeezing
plt.figure(figsize=(6,4))
plt.plot(t_list2, varX1, label='Var(X1)')
plt.plot(t_list2, varX2, label='Var(X2)')
plt.xlabel('Time')
plt.ylabel('Variance')
plt.title('Two‑Mode Squeezed Vacuum Dynamics')
plt.legend(); plt.grid(True)
plt.savefig('two_mode_squeezing.png', dpi=300)
plt.show()

In [ ]:
##CV Cluster States & Wigner

In [ ]:
# Parameters
N = 15
r = 1.0       # squeeze
g = 1.0       # CZ strength
xvec = np.linspace(-3,3,200)

# Initial vacuum
vac = basis(N,0)
psi = tensor(vac, vac)

# Apply squeezers
S1 = tensor(squeeze(N, r), qeye(N))
S2 = tensor(qeye(N), squeeze(N, r))
psi = S2 * (S1 * psi)

# CZ gate
X1 = (tensor(destroy(N), qeye(N)) + tensor(destroy(N).dag(), qeye(N)))/np.sqrt(2)
X2 = (tensor(qeye(N), destroy(N)) + tensor(qeye(N), destroy(N).dag()))/np.sqrt(2)
U_CZ = expm(1j * g * X1 * X2)
psi = U_CZ * psi

# Reduced states
rho1 = ptrace(psi.proj(), 0)
rho2 = ptrace(psi.proj(), 1)

# Wigner
W1 = wigner(rho1, xvec, xvec)
W2 = wigner(rho2, xvec, xvec)

In [ ]:
# Plot Wigner Functions
fig, axes = plt.subplots(1,2, figsize=(8,4))
im1 = axes[0].contourf(xvec, xvec, W1, 100, cmap='RdBu')
axes[0].set_title('Mode 1 Wigner')
axes[0].set_xlabel('x'); axes[0].set_ylabel('p')
fig.colorbar(im1, ax=axes[0])

im2 = axes[1].contourf(xvec, xvec, W2, 100, cmap='RdBu')
axes[1].set_title('Mode 2 Wigner')
axes[1].set_xlabel('x'); axes[1].set_ylabel('p')
fig.colorbar(im2, ax=axes[1])

plt.suptitle('2‑Mode CV Cluster Wigner Functions')
plt.savefig('cv_cluster_wigner.png', dpi=300)
plt.show()